In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

%matplotlib inline

# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')

**ПОДГОТОВКА ДАННЫХ**

In [2]:
# # Сортируем по дате и id-клиента
# train=pd.read_csv('/content/drive/My Drive/Data CupIT/CUP_IT_train_data.csv')
# train['id_data']=train['cif_id'].astype(str)+'_'+train['dlk_cob_date']
# train=train.sort_values('id_data')

Работаем с пропусками в данных

In [3]:
# miss_dict={}
# for column_name in train.columns:
#     miss_dict[column_name] = train[column_name].isnull().sum()
# miss_list_for_del=[]
# miss_list_for_update=[]
# for key in miss_dict:
#     if miss_dict[key]>=0.5*train.shape[0]:
#         miss_list_for_del.append(key)
#     elif miss_dict[key] != 0:
#         miss_list_for_update.append(key)
# # print(miss_list_for_del)
# # print(miss_list_for_update)
# print('List features for delete: ', miss_list_for_del) # список на удаление - слишком много пропусков
# print('List features for update: ', miss_list_for_update) # список на обновление - восстановить пропуски

Удаляем данные с числом пропусков более 50%

In [4]:
# train=train.drop(miss_list_for_del, axis=1)

Признаки с небольшим количеством пропусков заполняем в категориальных признаках Unknown, в количественном - медианой

In [5]:
# train['cu_education_level']=train['cu_education_level'].fillna('Unknown')
# train['cu_empl_area']=train['cu_empl_area'].fillna('Unknown')
# train['cu_empl_level']=train['cu_empl_level'].fillna('Unknown')
# train['ca_volume']=train['ca_volume'].fillna(train['ca_volume'].median())

**РАБОТА С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ**

In [6]:
# # Получение списка категориальных переменных
# s = (train.dtypes == 'object')
# object_cols = list(s[s].index)
# print((object_cols))

In [7]:
# train['big_city']=train['big_city'].astype('category')
# train['cu_education_level']=train['cu_education_level'].astype('category')
# train['cu_empl_level']=train['cu_empl_level'].astype('category')

**ОБРАБОТКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ - OHE HOT ENCODING**

In [8]:
# from sklearn.preprocessing import OneHotEncoder
# list_for_ohe=['big_city', 'cu_education_level']
# # Применим OHE к каждой колонке с категориальными данными
# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[list_for_ohe]))

# # One-hot encoding удаляет индекс; восстановим его
# OH_cols_train.index = train.index

# # Удалим категориальные признаки, заменив их на перекодированные
# num_X = train.drop(list_for_ohe, axis=1)

# # Add one-hot encoded columns to numerical features
# OH_X = pd.concat([num_X, OH_cols_train], axis=1)

In [9]:
# OH_X=OH_X.rename(columns={0:'big_city_OTH', 1:'big_city_SPB', 2:'big_city_MLN', 3:'big_city_MSK',
#                           4: 'cu_education_level_3',5: 'cu_education_level_2',6: 'cu_education_level_4',7: 'cu_education_level_1', 8: 'cu_education_level_Unknown'})

In [10]:
# traindf=OH_X.drop(columns=['id_data'], axis=1)

**MEAN TARGET**

Применим MEAN TARGET подход для перекодировки категориальных признаков с большим числом вариаций признака

In [11]:
# cu_empl_lev_meantarget={}
# for level in train['cu_empl_level'].unique():
#   cu_empl_lev_meantarget[level]=traindf[traindf['cu_empl_level']==level]['gi_smooth_3m'].mean()
# traindf['cu_empl_level']=traindf['cu_empl_level'].map(lambda x: cu_empl_lev_meantarget[x])
# traindf=traindf.rename(columns={'cu_empl_level':'cu_empl_level_mean_target'})

In [12]:
# cu_empl_area_meantarget={}
# for area in train['cu_empl_area'].unique():
#   cu_empl_area_meantarget[area]=traindf[traindf['cu_empl_area']==area]['gi_smooth_3m'].mean()

In [13]:
# traindf['cu_empl_area']=traindf['cu_empl_area'].map(lambda x: cu_empl_area_meantarget[x])
# traindf=traindf.rename(columns={'cu_empl_area':'cu_empl_area_mean_target'})

In [14]:
# traindf['cu_empl_level_mean_target']=traindf['cu_empl_level_mean_target'].astype('float64')

**ГОТОВИМ ЦЕЛЕВУЮ ПЕРЕМЕННУЮ**

In [15]:
# train=traindf
# # train=pd.read_csv('/content/drive/My Drive/Data CupIT/dataset.csv')

In [16]:
# # делаем сдвиг по каждому из пользователей
# train['target_gi_1m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-6)
# train['target_gi_2m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-7)
# train['target_gi_3m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-8)
# train['target_gi_4m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-9)
# train['target_gi_5m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-10)
# train['target_gi_6m']=train.groupby(['cif_id'])['gi_smooth_3m'].shift(-11)

СОКРАТИМ ДАТАСЕТ в 4 раза

In [17]:
# clients=train['cif_id'].unique()

In [18]:
# client_list=[]
# k=0
# for client in clients:
#   if k % 4 == 0:
#     client_list.append(client)
#   k += 1

ОТБЕРЕМ ДАННЫЕ ДЛЯ КАЖДОГО КЛИЕНТА ЗА ПЕРВЫЕ 6 МЕСЯЦЕВ и "ПРОТЯНЕМ" ЦЕЛЕВУЮ ПЕРЕМЕННУЮ НА ВСЕ СТРОКИ ПО КЛИЕНТУ

In [19]:
# fulldata=pd.DataFrame()
# for client in client_list:
#   subset=train[train['cif_id']==client].iloc[:6, :]
#   for i in range(1, len(subset)):
#     subset.iloc[i, [46,47,48,49,50,51]]=subset.iloc[0, [46,47,48,49,50,51]]
#   fulldata=pd.concat([fulldata, subset])

СОЗДАДИМ ЦЕЛЕВУЮ ПЕРЕМЕННУЮ

In [20]:
# fulldata['CLTV']=fulldata['target_gi_1m']+fulldata['target_gi_2m']+fulldata['target_gi_3m']+fulldata['target_gi_4m']+fulldata['target_gi_5m']+fulldata['target_gi_6m']
# fulldata=fulldata.dropna()

In [21]:
# X=fulldata.drop(columns=['cif_id','dlk_cob_date','target_gi_1m', 'target_gi_2m', 'target_gi_3m', 'target_gi_4m', 'target_gi_5m', 'target_gi_6m', 'CLTV'])
# y=fulldata['CLTV']

**ОТБОР ПРИЗНАКОВ**


In [22]:
# feature_names = np.array(X.columns)
# feature_scores = pd.DataFrame(index=feature_names)

In [23]:
# method_name = 'Random Forest'
# random_forest = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
# random_forest.fit(X, y)

# feature_scores[method_name] = random_forest.feature_importances_.reshape(-1, 1)
# feature_scores[method_name] /= feature_scores[method_name].max()

In [24]:
# feature_scores['F test'], _ = f_regression(X, y)
# feature_scores['F test'] /= feature_scores['F test'].max()

In [25]:
# method_name = 'Ridge'
# ridge = Ridge(normalize=True).fit(X, y)
# feature_scores[method_name] = np.abs(ridge.coef_).reshape(-1, 1)
# feature_scores[method_name] /= feature_scores[method_name].max()

In [26]:
# feature_scores['average_all_models']=(feature_scores['F test']+feature_scores['Ridge'])/2
# feature_scores=feature_scores.drop(columns=['Random Forest'], axis=1)
# feature_scores=feature_scores.sort_values('average_all_models', ascending=False)
# feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

ОТБЕРЕМ ПРИЗНАКИ, Имеющие показатель average_all_models свыше 0.03

In [27]:
features_select=['gi_smooth_3m','cur_quantity_mort', 'cur_quantity_pl', 'cur_quantity_cc', 'cc_cash_spend_c', 'active', 'cu_education_level_1',  'big_city_SPB', 'cur_quantity_mf', 
                 'big_city_MLN', 'standalone_nonpayroll_dc_f', 'cu_education_level_Unknown','standalone_dc_f', 'cu_education_level_3', 'big_city_OTH', 'cur_quantity_dc', 
                 'big_city_OTH', 'cu_education_level_2', 'big_city_MSK', 'ca_f', 'cur_quantity_deposits']

In [28]:
# X_train=X[features_select]

Разбиение данных на train и test


In [29]:
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.3, random_state=1)

**ОБУЧАЕМ МОДЕЛИ**

In [31]:
# from sklearn.linear_model import LinearRegression
# model1=LinearRegression(n_jobs=-1)
# model1.fit(X_train, y_train)
# preds1=model1.predict(X_valid)

In [32]:
# from sklearn.linear_model import Ridge
# model2=Ridge()
# model2.fit(X_train, y_train)
# preds2=model2.predict(X_valid)

In [33]:
# from sklearn.neighbors import KNeighborsRegressor
# model3=KNeighborsRegressor()
# model3.fit(X_train, y_train)
# preds3=model3.predict(X_valid)

In [34]:
# model4= RandomForestRegressor(n_estimators=500, max_depth=7, random_state=0, n_jobs=-1)
# model4.fit(X_train, y_train)
# preds4=model4.predict(X_valid)

In [35]:
# from sklearn.tree import DecisionTreeRegressor
# model5=DecisionTreeRegressor(max_depth=7)
# model5.fit(X_train, y_train)
# preds5=model5.predict(X_valid)

In [36]:
# from sklearn.linear_model import SGDRegressor
# model6=SGDRegressor()
# model6.fit(X_train, y_train)
# preds6=model6.predict(X_valid)

**МЕТРИКА КАЧЕСТВА**

In [37]:
# import numpy as np

# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [38]:
# MAPE1=mean_absolute_percentage_error(preds1, y_valid)
# MAPE2=mean_absolute_percentage_error(preds2, y_valid)
# MAPE3=mean_absolute_percentage_error(preds3, y_valid)
# MAPE4=mean_absolute_percentage_error(preds4, y_valid)
# MAPE5=mean_absolute_percentage_error(preds5, y_valid)
# MAPE6=mean_absolute_percentage_error(preds6, y_valid)
# print('MAPE for LR: ', MAPE1)
# print('MAPE for Ridge: ', MAPE2)
# print('MAPE for KNN: ', MAPE3)
# print('MAPE for RF: ', MAPE4)
# print('MAPE for DTR: ', MAPE5)
# print('MAPE for SGD: ', MAPE6)

Дополнительная метрика качества - MAE

In [39]:
# MAE1=mean_squared_error(y_valid, preds1)
# MAE2=mean_squared_error(y_valid, preds2)
# MAE3=mean_squared_error(y_valid, preds3)
# MAE4=mean_squared_error(y_valid, preds4)
# MAE5=mean_squared_error(y_valid, preds5)
# MAE6=mean_squared_error(y_valid, preds6)
# print('MAE for LR: ', MAE1)
# print('MAE for Ridge: ', MAE2)
# print('MAE for KNN: ', MAE3)
# print('MAE for RF: ', MAE4)
# print('MAE for DTR: ', MAE5)
# print('MAE for SGD: ', MAE6)

ПО МЕТРИКАМ КАЧЕСТВА ЛУЧШАЯ МОДЕЛЬ - RandomForestRegressor.
Сохраним ее для будущих предсказаний.

In [40]:
import joblib
filename = 'finalized_model.sav'
# joblib.dump(model4, filename)

**ПРЕДСКАЗАНИЕ МОДЕЛИ**

In [41]:
test=pd.read_csv('CUP_IT_test_data.csv')

**Сортируем данные по датам**

In [42]:
test['id_data']=test['cif_id'].astype(str)+'_'+test['dlk_cob_date']
test=test.sort_values('id_data')

Удаляем пропуски

In [43]:
miss_list_for_del=['cc_balance', 'cl_balance', 'ml_balance', 'pl_balance', 'td_volume', 'sa_volume', 'mf_volume']
test=test.drop(miss_list_for_del, axis=1)

Заменяем пропуски

In [44]:
test['cu_education_level']=test['cu_education_level'].fillna('Unknown')
test['cu_empl_area']=test['cu_empl_area'].fillna('Unknown')
test['cu_empl_level']=test['cu_empl_level'].fillna('Unknown')
test['ca_volume']=test['ca_volume'].fillna(test['ca_volume'].median()) 

Обработка категориальных переменных

In [45]:
# Получение списка категориальных переменных
s = (test.dtypes == 'object')
object_cols = list(s[s].index)
print((object_cols))

['dlk_cob_date', 'big_city', 'cu_education_level', 'cu_empl_area', 'cu_empl_level', 'id_data']


In [46]:
test['big_city']=test['big_city'].astype('category')
test['cu_education_level']=test['cu_education_level'].astype('category')
test['cu_empl_level']=test['cu_empl_level'].astype('category')

**ONE HOT ENCODING для тестовых данных**

In [47]:
from sklearn.preprocessing import OneHotEncoder
list_for_ohe=['big_city', 'cu_education_level']
# Применим OHE к каждой колонке с категориальными данными
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(test[list_for_ohe]))

# One-hot encoding удаляет индекс; восстановим его
OH_cols_train.index = test.index

# Удалим категориальные признаки, заменив их на перекодированные
num_X = test.drop(list_for_ohe, axis=1)

# Add one-hot encoded columns to numerical features
OH_X = pd.concat([num_X, OH_cols_train], axis=1)
OH_X=OH_X.rename(columns={0:'big_city_OTH', 1:'big_city_SPB', 2:'big_city_MLN', 3:'big_city_MSK',
                          4: 'cu_education_level_3',5: 'cu_education_level_2',6: 'cu_education_level_4',7: 'cu_education_level_1', 8: 'cu_education_level_Unknown'})
testdf=OH_X.drop(columns=['id_data'], axis=1)

**MEAN TARGET для тестовых данных**

In [48]:
test_cu_empl_lev_meantarget={}
for level in test['cu_empl_level'].unique():
  test_cu_empl_lev_meantarget[level]=testdf[testdf['cu_empl_level']==level]['gi_smooth_3m'].mean()
testdf['cu_empl_level']=testdf['cu_empl_level'].map(lambda x: test_cu_empl_lev_meantarget[x])
testdf=testdf.rename(columns={'cu_empl_level':'cu_empl_level_mean_target'})

In [49]:
test_cu_empl_area_meantarget={}
for area in test['cu_empl_area'].unique():
  test_cu_empl_area_meantarget[area]=testdf[testdf['cu_empl_area']==area]['gi_smooth_3m'].mean()

In [50]:
testdf['cu_empl_area']=testdf['cu_empl_area'].map(lambda x: test_cu_empl_area_meantarget[x])
testdf=testdf.rename(columns={'cu_empl_area':'cu_empl_area_mean_target'})
testdf['cu_empl_level_mean_target']=testdf['cu_empl_level_mean_target'].astype('float64')

**ГОТОВИМ ТЕСТОВЫЕ ДАННЫЕ и ДЕЛАЕМ ПРЕДСКАЗАНИЕ**

In [54]:
testdata=testdf[features_select]
filename = 'finalized_model.sav'
loaded_model = joblib.load(filename)
prediction = loaded_model.predict(testdata)
prediction=pd.DataFrame(prediction)
prediction['cif_id']=test['cif_id']
prediction=prediction.rename(columns={0:'CLTV'})
prediction['CLTV_2']=prediction['CLTV']
prediction=prediction.drop(columns=['CLTV']).rename(columns={'CLTV_2':'CLTV'})
prediction_final=prediction.groupby(by='cif_id').mean()
prediction_final=prediction_final.reset_index()

In [ ]:
prediction_final.to_csv('CUP_IT_predictions.csv', index=False)